# The making of...

Het maken van de verschillende programma's voor de microbit ging niet zonder hindernissen.
Ik beschrijf hier enkele van de problemen die ik tegengekomen ben, en de manier waarop ik die opgelost heb.

## De radio-logger

Het iot:bit bordje bevat een ESP8266 die via de seriële verbinding met de microbit communiceert.
Een probleem is dat de microbit maar één seriële verbinding heeft.
Deze wordt gewoonlijk gebruikt voor de communicatie met de host.
Via die verbinding kun je dan de uitvoer van print-opdrachten zien, en kun je eventueel REPL-opdrachten geven.
Maar: als je die verbinding nodig hebt voor de communicatie met de ESP8266, dan kun je die niet gebruiken voor print-opdrachten, wat het foutzoeken op de microbit een stuk lastiger maakt.

Mijn eerste oplossing was om de seriële verbinding van de microbit snel te schakelen tussen de host-verbinding en de ESP8266. Maar al snel bleek dat de USB-aansluiting van het iot:bit-borcdje niet verbonden is met de seriële verbinding van de microbit. (Je kunt de USB-aansluiting op de microbit zelf niet gebruiken, omdat die niet voldoende stroom levert voor de ESP8266.) Dit werkt dus niet.

De tweede, en werkende, oplossing is om de radio-verbinding van de microbit te gebruiken voor *logging*, om daarmee tekst-uitvoer vanuit het microbit-programma mogelijk te maken.
De iot:bit microbit stuurt teksten via de radio, die door een extra microbit ontvangen worden.
Deze laatste geeft de teksten weer via print-opdrachten naar de host.
Dit heeft het proces van foutzoeken aanzienlijk versneld.

Een open vraag voor mij was het AT-protocol tussen de microbit en de ESP8266: omdat dit niet compleet gedocumenteerd is, moest ik dingen uitproberen. Ik wil dan weten wat er gebeurt op de verbinding tussen de microbit en de ESP8266. Dankzij de *logging* via de radio was dat goed te volgen.

(Deze radio-aanpak werkt helaas niet voor de ingebouwde print-opdrachten, bijvoorbeeld van de foutmeldingen: die moet je zien op te maken uit de letters die op het microbit-display voorbij scrollen.)


(figuur bij deze uitleg...)

## JSON

Voor het versturen van de sensordata via MQTT maken we gebruik van JSON (zoals in veel MQTT toepassingen gebruikelijk is). Python heeft een module `json` om een Python waarde (bijv. dictionary) om te zetten in een JSON object, en omgekeerd. Deze is gewoonlijk ook in microPython beschikbaar. Maar deze blijkt niet aanwezig te zijn in microPython voor de micro:bit...

De sensor-en actuator-data gebruiken maar een deel van de mogelijkheden van JSON. Ik heb functies gemaakt die in elk geval deze mogelijkheden ondersteunt, dat is wat eenvoudiger dan een complete JSON-implementatie. Het omzetten van Python naar JSON is vrij eenvoudig: je komt met het omzetten naar string-notatie een heel eind. Het omgekeerde is lastiger: je hebt dan een JSON-parser nodig. Nu is compiler-techniek mijn oude vak, dus dat was redelijk snel te doen. Weer een blokkade opgeruimd.

## Escape-codes

In de AT-opdrachten voor de ESP-WiFi module kunnen strings voorkomen, zoals de naam van het WiFi netwerk, het wachtwoord e.d..
In het AT-formaat moeten die strings tussen dubbele quotes (`"`) geplaatst worden, zie bijv.:

```
AT+MQTTPUB=0,"topic","\"{\"sensor\":012}\"",1,0
```

Tot nu toe niets bijzonders.

In de string zelf kunnen ook dubbele quotes (`"`) voorkomen: dat is bijvoorbeeld het geval met de JSON-inhoud van MQTT berichten, bij de "publish" opdracht. De gebruikelijke manier om dit op te lossen is om deze quotes van een escape-code te voorzien: je zet dan het teken `\` voor het betreffende teken: `\"`.
(Je moet dan ook weer de mogelijkheid hebben om dat teken in de string weer te geven, als `\\`.)
Op deze manier is ondubbelzinnig duidelijk welke tekens tot de string zelf behoren, en welke bij de omvattende AT-opdracht horen.

Maar: dit bleek nog steeds foutmeldingen op te leveren van de betreffende AT-opdracht. (Lang leve de radio-logger...).

Bij nader lezen van de documentatie bleek dat ook **komma's** die voorkomen in een AT-string, **voorzien moeten worden van een escape-code**. (Dit is *wel* bijzonder: zoals gezegd, is met bovenstaande escape-regels al duidelijk of die comma tot de string behoort of niet.)

* https://espressif-docs.readthedocs-hosted.com/projects/esp-at/en/release-v2.1.0.0_esp8266/AT_Command_Set/index.html#at-command-types

Toen dat aangepast was, bleek de MQTT "publish" opdracht voor JSON-berichten ineens te werken.

Hieruit blijkt maar weer: documentatie goed lezen, en er niet vanuit gaan dat de programmeurs zich aan de "normale regels" houden. (Maar: het lijkt erop dat als je een programmeerfout maar documenteert, het geen fout meer is.)